In [163]:
import pandas as pd

import datetime
from datetime import timedelta

from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import seasonal_decompose 

from statsmodels.tsa.holtwinters import SimpleExpSmoothing   

from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [164]:
path_occ = '~/Downloads/park-ride-rapperswil.csv'
path_tickets = '~/Downloads/parkrail-sale-app.csv'

In [165]:
true_occ = pd.read_csv(path_occ, sep=';')
tickets = pd.read_csv(path_tickets, sep=';')

In [166]:
tickets['start'] = pd.to_datetime(tickets['start'])
tickets['end'] = pd.to_datetime(tickets['end'])

In [167]:
tickets['start'] = tickets['start'].apply(lambda x: x.replace(tzinfo=None))
tickets['end'] = tickets['end'].apply(lambda x: x.replace(tzinfo=None))

In [168]:
TOTAL_SPACE = 180

In [169]:
#############################################
# Tickets Processing

In [170]:
tickets = tickets.sort_values(by="start") 

In [171]:
def rounder(t):
    if t.minute >= 30:
        try:
            return t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
        except ValueError:
            #return t.replace(day=t.day+1, second=0, microsecond=0, minute=0, hour=0)
            t = t.replace(second=0, microsecond=0, minute=0, hour=0)
            t = t+timedelta(hours=24)
            return t
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [172]:
d = {}
for row in range(len(tickets)):
    
    td = tickets.iloc[row][1] - tickets.iloc[row][0]
    hours = td.total_seconds() / 3600
    
    start = rounder(tickets.iloc[row][0])
    for hour in range(int(hours+1)):
        if start+timedelta(hours=hour) in d:
            d[start+timedelta(hours=hour)] += 1
        else:
            d[start+timedelta(hours=hour)] = 1

In [173]:
tickets_conv = pd.DataFrame(d.items(), columns=['date', 'occupancy_rate'])

In [175]:
# Occupancy dataset from ticket sales
tickets_conv['occupancy_rate'] = (tickets_conv['occupancy_rate'] / TOTAL_SPACE)*100

In [177]:
#############################################

In [178]:
#############################################
# True occupancy processing

In [179]:
true_occ['Datum'] = pd.to_datetime(true_occ['Datum'])

In [180]:
true_occ = true_occ.rename(columns={'Datum': 'date', 'BELEGUNGSQUOTE (%)': 'occupancy_rate'})

In [181]:
true_occ['date'] = true_occ['date'].apply(lambda x: x.replace(tzinfo=None))

In [182]:
true_occ = true_occ.sort_values(by="date")
true_occ = true_occ.groupby("date").mean()

In [183]:
#############################################

In [184]:
#############################################
# Add Gaussian Noise to simulate user inputs

In [185]:
mu, sigma = 0, 5 
# creating a noise with the same dimension as the dataset (2,2) 
noise = np.random.normal(mu, sigma, [len(true_occ),1]) 

user_input = true_occ + noise
##########################################

In [186]:
#############################################
# Create training data

In [ ]:
#############################################
# Fill missing dates

In [188]:
tickets_train = tickets_conv[tickets_conv['date']>=user_input.index[0]]
tickets_train = tickets_train.groupby("date").mean()

user_input_train = user_input.loc[:tickets_train.index[-1]]

In [189]:
user_input_train = user_input_train.reset_index()
for i in range(1,len(user_input_train)):
    if user_input_train.iloc[i]['date'] != user_input_train.iloc[i-1]['date']+timedelta(hours=1):
        line = pd.DataFrame({"date": user_input_train.iloc[i-1]['date']+timedelta(hours=1),"occupancy_rate": user_input_train["occupancy_rate"].quantile(.25)}, index=[i])
        user_input_train = pd.concat([user_input_train.iloc[:i], line, user_input_train.iloc[i:]]).reset_index(drop=True)


In [190]:
tickets_train = tickets_train.reset_index()
for i in range(1,len(tickets_train)):
    if tickets_train.iloc[i]['date'] != tickets_train.iloc[i-1]['date']+timedelta(hours=1):
        line = pd.DataFrame({"date": tickets_train.iloc[i-1]['date']+timedelta(hours=1),"occupancy_rate": tickets_train.iloc[i-1]['occupancy_rate']}, index=[i])
        tickets_train = pd.concat([tickets_train.iloc[:i], line, tickets_train.iloc[i:]]).reset_index(drop=True)

In [191]:
user_input_train = user_input_train.groupby("date").mean()
tickets_train = tickets_train.groupby("date").mean()

In [194]:
#############################################

In [198]:
train_data = user_input_train*0.7+user_input_train*0.3